In [ ]:
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel("data/QUARTERLY-1.xls")
df['date'] = pd.PeriodIndex(df['Date'], freq='Q')
df.set_index('date', inplace=True)
df = df[["CPINSA","Date"]]
df

In [ ]:
#3a
plt.figure(figsize=(8, 5))
plt.plot(df.index.to_timestamp(), df['CPINSA'], marker='o')
plt.title('Quarterly Data')
plt.xlabel('Date')
plt.ylabel('TB3mo')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# does not look staionary 

In [ ]:
#3b
fig = plot_acf(df['CPINSA'], lags=25)
plt.show()

In [ ]:
plot_pacf(df['CPINSA'], lags=25)
plt.show()


In [ ]:
#c 
df["log_CPINSA"] = np.log((df["CPINSA"] / df["CPINSA"].shift(1)))
df

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(df.index.to_timestamp(), df['log_CPINSA'], marker='o')
plt.title('Quarterly Data')
plt.xlabel('Date')
plt.ylabel('TB3mo')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# there was a dip but it seems to be stationary given that there is no strong trend 

In [ ]:
#3d
fig = plot_acf(df['log_CPINSA'].dropna(), lags=25)
plt.show()

In [ ]:
plot_pacf(df['log_CPINSA'].dropna(), lags=25)
plt.show()


In [ ]:
#3e
df["log_CPINSA4"] = np.log((df["CPINSA"] / df["CPINSA"].shift(4)))
df

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(df.index.to_timestamp(), df['log_CPINSA4'], marker='o')
plt.title('Quarterly Data')
plt.xlabel('Date')
plt.ylabel('TB3mo')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# there was a dip but it seems to be stationary given that there is no strong trend 

In [ ]:
#f
fig = plot_acf(df['log_CPINSA4'].dropna(), lags=25)
plt.show()

In [ ]:
fig = plot_pacf(df['log_CPINSA4'].dropna(), lags=25)
plt.show()

In [ ]:
#3g
res = AutoReg(df['log_CPINSA4'].dropna(), lags =5).fit()
print(res.summary())

In [ ]:
y_true = res.model.endog[res.model._hold_back:]  
y_pred = res.fittedvalues


ssr = np.sum((y_true - y_pred) ** 2)

tss = np.sum((y_true - np.mean(y_true)) ** 2)

n = len(y_true)
k = res.df_model + 1  

r2 = 1 - ssr / tss
r2_adj = 1 - (ssr / (n - k)) / (tss / (n - 1))

print("R^2:", round(r2, 4))
print("Adjusted R^2:", round(r2_adj, 4))

In [ ]:
arma_mod = ARIMA(df['log_CPINSA4'].dropna(), order=(0, 0, 10)).fit()
print(arma_mod.summary())

In [ ]:
arma_mod = ARIMA(df['log_CPINSA4'].dropna(), order=(5, 0, 10)).fit()
print(arma_mod.summary())

In [ ]:
arma_mod = ARIMA(df['log_CPINSA4'].dropna(), order=(6, 0, 7)).fit()
print(arma_mod.summary())

In [ ]:
#h

In [ ]:
# looking at the AIC the MA(10) seems to prefrom the best

In [ ]:
#i
# Step 1: Extract quarter from datetime index
df["quarter"] = df.index.quarter

# Step 2: Create dummy variables with 0/1 values
quarter_dummies = pd.get_dummies(df["quarter"], prefix="Q", drop_first=True).astype(int)

# Step 3: Concatenate with main DataFrame
df = pd.concat([df, quarter_dummies], axis=1)
df

In [ ]:
X = df[[col for col in df.columns if col.startswith("Q_")]]
X = sm.add_constant(X)
y = df["log_CPINSA4"]

model = sm.OLS(y, X, missing='drop').fit()
print(model.summary())


In [ ]:
#j
df["residuals"] = model.resid
plt.figure(figsize=(10, 5))
plt.plot(df.index.to_timestamp(), df["residuals"], marker='o', linestyle='-')
plt.axhline(0, color='red', linestyle='--')
plt.title("Regression Residuals: log(CPINSA_t / CPINSA_{t-1}) ~ Quarterly Dummies")
plt.xlabel("Date")
plt.ylabel("Residual")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# the residuals apear to be staionary 

In [ ]:
#k
fig = plot_acf(df['log_CPINSA4'].dropna(), lags=25)
plt.show()

In [ ]:
fig = plot_pacf(df['log_CPINSA4'].dropna(), lags=25)
plt.show()

In [ ]:
# the residuals seam to be autocorrolated by round 10 periods